In [1]:
import numpy as np 
import pandas as pd
import requests
from IPython.display import Image, HTML, display
import warnings
def warns(*args,**kwargs): pass
warnings.warn=warns
import os
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
countVec = CountVectorizer(max_features=10000, stop_words='english')
ps = PorterStemmer()

In [2]:
dfmovies = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
dfcredits = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")

In [3]:
dfmovies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
dfcredits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
dfcombine = dfmovies.merge(dfcredits, on='title')
dfcombine.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [5]:
features_cols = ['id', 'title', 'original_title', 'tagline', 'genres', 'keywords', 'cast', 'crew', 'overview']
df_final = dfcombine[features_cols]
df_final.head()

,id,title,original_title,tagline,genres,keywords,cast,crew,overview
0,19995,Avatar,Avatar,Enter the World of Pandora.,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,Spectre,A Plan No One Escapes,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,The Dark Knight Rises,The Legend Ends,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",Following the death of District Attorney Harve...
4,49529,John Carter,John Carter,"Lost in our world, found in another.","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","John Carter is a war-weary, former military ca..."


In [6]:
df_final.isnull().sum()

id                  0
title               0
original_title      0
tagline           844
genres              0
keywords            0
cast                0
crew                0
overview            3
dtype: int64

In [9]:
df_final['tagline'] = df_final['tagline'].fillna('unknown')
df_final.dropna(inplace=True)
df_final.isnull().sum()

id                0
title             0
original_title    0
tagline           0
genres            0
keywords          0
cast              0
crew              0
overview          0
dtype: int64

In [10]:
def colData_Transform(obj):
    lst = []
    for i in ast.literal_eval(obj):
        lst.append(i['name'].replace(" ","").lower())
    return lst

def fetch_cast_data(obj):
    lst = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 5:
            lst.append(i['name'].replace(" ","").lower())
            counter += 1
    return lst

def get_crew_data(obj):
    lst = []
    for x in ast.literal_eval(obj):
        if x['job'] == "Director":
            lst.append(x['name'].replace(" ","").lower())

    return lst

In [11]:
df_final['tags'] = df_final['overview'].apply(lambda x: x.lower() and x.split() )
df_final['tags'] = df_final['tags'] + df_final['genres'].apply(colData_Transform)
df_final['tags'] = df_final['tags'] + df_final['keywords'].apply(colData_Transform)
df_final['tags'] = df_final['tags'] + df_final['cast'].apply(fetch_cast_data)
df_final['tags'] = df_final['tags'] + df_final['crew'].apply(get_crew_data)
df_final['tags'] = df_final['tags'] + df_final['title'].apply(lambda x: x.lower() and x.split() )

In [12]:
for x, xRow in df_final.iterrows():
    if (xRow['original_title'] != xRow['title'] ):
        df_final.at[x, 'tags'] = xRow['tags'] + xRow['original_title'].lower().split()

In [13]:
for x, xRow in df_final.iterrows():
    if (xRow['tagline'] != 'unknown'):
        df_final.at[x, 'tags'] = xRow['tags'] + xRow['tagline'].lower().split()

In [14]:
df_new_final = df_final[['id', 'title', 'tags']]
df_new_final['tags'] = df_new_final['tags'].apply(lambda x: " ".join(x).lower())
df_new_final['tags'] = df_new_final['tags'].str.lower()
df_new_final.head()

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [15]:
def xStem(txt):
    y = []
    for x in txt.split():
        y.append(ps.stem(x))
    return " ".join(y)

In [16]:
df_new_final['tags'] = df_new_final['tags'].apply(xStem)
dataVectors = countVec.fit_transform(df_new_final['tags']).toarray()
similarity = cosine_similarity(dataVectors)

In [17]:
similarity

array([[1.        , 0.11013346, 0.06598588, ..., 0.05458155, 0.        ,
        0.        ],
       [0.11013346, 1.        , 0.06419407, ..., 0.05309942, 0.03241019,
        0.01785714],
       [0.06598588, 0.06419407, 1.        , ..., 0.02120949, 0.        ,
        0.        ],
       ...,
       [0.05458155, 0.05309942, 0.02120949, ..., 1.        , 0.03212463,
        0.05309942],
       [0.        , 0.03241019, 0.        , ..., 0.03212463, 1.        ,
        0.08102547],
       [0.        , 0.01785714, 0.        , ..., 0.05309942, 0.08102547,
        1.        ]])

In [18]:
def recommend_movies_names(xMovie):
    movie_index = df_new_final[df_new_final["title"].str.lower() == xMovie.lower()].index[0]
    distances = similarity[movie_index]
    listofMovies = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in listofMovies:
        print(df_new_final.iloc[i[0]]["title"])

In [27]:
movies_list = {"Avatar"}
for x in movies_list:
    print("-"*20, "Recommendations for [", x,"]", "-"*20)
    recommend_movies_names(x)

-------------------- Recommendations for [ Avatar ] --------------------
Aliens
Aliens vs Predator: Requiem
Aliens in the Attic
Titan A.E.
Independence Day


In [28]:
movies_list = {"Batman"}
for x in movies_list:
    print("-"*20, "Recommendations for [", x,"]", "-"*20)
    recommend_movies_names(x)

-------------------- Recommendations for [ Batman ] --------------------
Batman
Batman & Robin
Batman Begins
Batman Returns
The Dark Knight


In [29]:
movies_list = {"Aliens"}
for x in movies_list:
    print("-"*20, "Recommendations for [", x,"]", "-"*20)
    recommend_movies_names(x)

-------------------- Recommendations for [ Aliens ] --------------------
Alien
Alien³
Aliens vs Predator: Requiem
Meet Dave
Aliens in the Attic
